In [1]:
from data_utils import preprocessDataset
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [2]:
train_dataset, test_dataset_original = preprocessDataset(f'/media/data_dump/Mohit/facct23_samplebias_data/run_1/adult/raw/betaDatasets/imbalance_1.0_1.0', f'/media/data_dump/Mohit/facct23_samplebias_data/run_1/adult/raw/original_test.csv', 'adult')

In [3]:
from sklearn.neural_network import MLPClassifier
import math
from torch import nn
import torch

In [4]:
data_shape = train_dataset.features.shape

In [5]:
data_shape

(30162, 96)

In [6]:
from skorch import NeuralNetClassifier, NeuralNet
import skorch

In [34]:
class MyModule(nn.Module):
    def __init__(self, data_shape, nonlin=nn.ReLU()):
        super().__init__()
        # -1 for sensitive attribute removal (group-blind training)
        num_units = math.ceil((2*data_shape[0])/(data_shape[1] - 1))
        self.dense0 = nn.Linear(data_shape[1] - 1, num_units)
        self.nonlin = nonlin
        #self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(num_units, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, data, sample_weight, **kwargs):
        X = data
        X = self.nonlin(self.dense0(X))
        X = self.softmax(self.output(X))
        return X

class MyNet(NeuralNet):
    def __init__(self, *args, criterion__reduce=False, **kwargs):
        # make sure to set reduce=False in your criterion, since we need the loss
        # for each sample so that it can be weighted
        super().__init__(*args, criterion__reduce=criterion__reduce, **kwargs)

    def get_loss(self, y_pred, y_true, X, *args, **kwargs):
        # override get_loss to use the sample_weight from X
        loss_unreduced = super().get_loss(y_pred, y_true, X, *args, **kwargs)
        sample_weight = skorch.utils.to_tensor(X['sample_weight'], device=self.device)
        loss_reduced = (sample_weight * loss_unreduced).mean()
        return loss_reduced

In [35]:
net = MyNet(
    MyModule(data_shape),
    criterion=nn.NLLLoss,
    optimizer=torch.optim.SGD,
    train_split=None,
    max_epochs=10,
    lr=0.01,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    device=0
)

In [36]:
X = train_dataset.features[:,:-1]
y = train_dataset.labels.ravel()

In [37]:
X = {'data': X,
    'sample_weight': [1]*len(y)}

In [38]:
net.fit(X, y)

RuntimeError: expected scalar type Float but found Double

In [ ]:
#from folktables import ACSIncome, ACSDataSource

In [8]:
#data_source = ACSDataSource(survey_year='2018', horizon='5-Year', survey='person', #root_dir='/media/data_dump/Mohit/facct23_samplebias_data/large_adult_source/')

In [ ]:
#acs_data = data_source.get_data(download=True)